In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import kagglegym
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
with pd.HDFStore("../input/train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

In [ ]:
env = kagglegym.make()
observation = env.reset()

In [ ]:
observation.train.head()

In [ ]:
list(observation.train.columns.values)

In [ ]:
observation.features.head()

In [ ]:
target = observation.train["y"].values

In [ ]:
def count_missing(df):
    stats={}
    for x in range(len(df.columns)):        
        stats[df.columns[x]]=df[df.columns[x]].isnull().sum()/len(df[df.columns[x]])*100
    return stats

In [ ]:
res=count_missing(df)

In [ ]:
low_err={k:v for (k,v) in res.items() if v<0.5}
del low_err['id']
del low_err['timestamp']
del low_err['y']
print((low_err),low_err.values())

In [ ]:
observation.train = observation.train.interpolate()

In [ ]:
observation.train

In [ ]:

features = observation.train[list(low_err.keys())]
features = features.fillna(0)

In [ ]:
features.shape

In [ ]:
features = features.values

In [ ]:
from sklearn import datasets, linear_model

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
my_regression = regr.fit(features, target)

In [ ]:
print(my_regression.score(features, target))

In [ ]:
observation.features.head()

In [ ]:
observation.target.head()

In [ ]:
test_features = observation.features[list(low_err.keys())].values

In [ ]:
prediction = my_regression.predict(test_features)

In [ ]:
prediction

In [ ]:
action = observation.target

In [ ]:
action['y'] = prediction

In [ ]:
observation, reward, done, info = env.step(action)

In [ ]:
reward

In [ ]:
done

In [ ]:
info

In [ ]:
while True: 
    observation.features = observation.features.interpolate()
    test_features = observation.features[list(low_err.keys())].values
    prediction = my_regression.predict(test_features)
    action = observation.target
    action['y'] = prediction
    
    timestamp = observation.features["timestamp"][0]
    
    if timestamp % 100 == 0:
        print("Timestamp #{}".format(timestamp))
        print(prediction)

    observation, reward, done, info = env.step(action)
    if done:        
        break

In [ ]:
info["public_score"]